In [1]:
from textwrap import wrap
from gensim.models import word2vec
import gensim
import pandas as pd
import numpy as np
import re
import string

from sklearn import metrics

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# для сравнения с линейной моделью
from sklearn.linear_model import LogisticRegression

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')

In [2]:
fname_train = r'linear_train.txt'
train = pd.read_csv(fname_train, names=['Word', 'Label'])
fname_test = r'linear_test.txt'
test = pd.read_csv(fname_test, names=['Word'])

In [4]:
train['Upper'] = train['Word'].str.istitle()
test['Upper'] = test['Word'].str.istitle()

train['2end'] = train['Word'].str[-2:].str.lower()
test['2end'] = test['Word'].str[-2:].str.lower()

In [6]:
surname_list_train = list(train['Word'].values)
surname_list_test = list(test['Word'].values)
# укажите количество символов, на которое будем разбивать слова
n = 3

surname_tokens = []
tokens_set = set()
for surname in surname_list_train + surname_list_test:
    tmp = wrap(surname, n)
    surname_tokens.append(tmp)

In [7]:
size_model = 25
model = gensim.models.Word2Vec(surname_tokens,
                               sg=1,
                               size=size_model,
                               workers=8,
                               min_count=1,
                               window=6,
                               iter=100,
#                               seed=42
                               )

In [8]:
surname_vectors = []
for sur in surname_tokens:
    summ = 0
    for i in range(len(sur)):
        summ += model[sur[i]]
    surname_vectors.append(summ/len(sur))
surname_vectors = (np.array(surname_vectors)).transpose()

In [9]:
train

,Word,Label,Upper,2end
0,Аалтонен,1,True,ен
1,Аар,0,True,ар
2,Аарон,0,True,он
3,ААРОН,0,False,он
4,Аарона,0,True,на
...,...,...,...,...
101403,Ёлкин,1,True,ин
101404,ёлкой,0,False,ой
101405,ёлок,0,False,ок
101406,ёлочкой,0,False,ой


In [11]:
df_sum = pd.concat([train, test], ignore_index=True)

In [12]:
for col_name, obj in zip(range(size_model), surname_vectors):
    df_sum[str(col_name)] = obj

In [13]:
def bigl(Famils):
    num = []
    for els in Famils:
        if els[0] in 'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЭЮЯ':
            num.append(1)
        else:
            num.append(0)
    return(np.array(num))


def slogs(Famils):
    num = []
    j = 0
    let = 'аоеёуыяэюи'
    for els in Famils:
        for lets in els:
            if lets in let or lets in let.upper():
                j += 1
        num.append(j)
        j = 0
    return(np.array(num))


def lens(Famils):
    num = []
    for els in Famils:
        num.append(len(els))
    return(np.array(num))

In [14]:
for ds in [df_sum]:
    ds['bigl'] = bigl(ds.Word)
    ds['slogs'] = slogs(ds.Word)
    ds['lens'] = lens(ds.Word)

In [21]:
skonca = 6


def grama(Famils, gr):
    num = []
    for els in Famils:
        if gr in els.lower()[-skonca:]:
            num.append(1)
        else:
            num.append(0)
    return(np.array(num))

In [22]:
def grama_end(Famils, gr):
    num = []
    for els in Famils:
        if gr in els.lower()[-len(gr):]:
            num.append(1)
        else:
            num.append(0)
    return(np.array(num))

In [23]:
good_n_grams = ['об', 'сб', 'кс', 'им', 'ич', 'ок', 'ву', 'ию', 'ие', 'эк', 'св',
                'бе', 'сп', 'ые', 'ия', 'мм', 'уп', 'ты', 'рр', 'уш', 'эн', 'рт',
                'ур', 'ах', 'ан', 'са', 'ре', 'сн', 'ад', 'ей', 'нд', 'эй', 'же',
                'юк', 'ло', 'ар', 'вс', 'ск', 'ме', 'бу', 'ши', 'ле', 'нв', 'дд',
                'уй', 'ер', 'во', 'ий', 'ед', 'ян', 'рл', 'иг', 'лэ', 'ни', 'рк',
                'вк', 'ну', 'ел', 'вн', 'лс', 'се', 'йе', 'оп', 'ов', 'аб', 'ва',
                'ен', 'ес', 'рн', 'тт', 'ол', 'шк', 'пи', 'вы', 'мп', 'ки', 'тк',
                'су', 'да', 'ал', 'ве', 'ин', 'нк', 'тн', 'рс', 'ст', 'ке', 'дс',
                'ку', 'лл', 'ри', 'нь', 'ик', 'сл', 'си', 'пп', 'де', 'ох', 'из',
                'ях', 'оу', 'ма', 'ем', 'рд', 'нс', 'ам', 'зк', 'но', 'аж', 'рх',
                'ак', 'чк', 'ра', 'ай', 'йл', 'од', 'хо', 'ти', 'по', 'за', 'ос',
                'еч', 'сс', 'йт', 'рь', 'мл', 'га', 'бл', 'ии', 'ит', 'ег', 'ми',
                'он', 'пр', 'ау', 'та', 'ко', 'ев', 'ро', 'уб', 'со', 'ек', 'кт',
                'тр', 'бо', 'ой', 'ор', 'фо', 'ка', 'го', 'ци', 'ль', 'ат', 'ви',
                'лд', 'бр', 'ил', 'ом', 'ул', 'мс', 'ыв', 'нг', 'йн', 'вь', 'дж',
                'ба',   'зу'
                ]

In [24]:
good_ends = ['цкий',  'ский', 'цкая', 'ская',
             'нен', 'ына', 'ина', 'ёва', 'ева', 'ова',
             'ым',  'на', 'ых', 'их', 'ёв']

In [25]:
for gram in good_ends:
    df_sum[gram] = grama_end(list(df_sum.Word), gram)

In [26]:
for gram in good_n_grams:
    df_sum[gram] = grama(list(df_sum.Word), gram)

In [28]:
df_sum["count_punctuations"] = df_sum["Word"].apply(
    lambda x: len([c for c in str(x) if c in string.punctuation]))

In [29]:
df_sum['Vow'] = df_sum['Word'].apply(lambda x: len(
    re.findall('[уеыаоэяию]', x, re.IGNORECASE)))
df_sum['noVow'] = df_sum['lens']-df_sum['Vow']
df_sum["has_digit"] = df_sum.Word.map(lambda x: 1 if len(
    [int(s) for s in set(x) if s.isdigit()]) > 0 else 0)

In [30]:
train = df_sum[df_sum.index < len(list(train.index))]
test = df_sum[df_sum.index >= len(list(train.index))]

In [31]:
col = [c for c in train if c not in ['Word', 'Label']]
col2 = [c for c in test if c not in ['Word']]

In [32]:
train_full = train[col]
test_full = test[col2]

In [33]:
train_full.shape

(101408, 216)

In [34]:
train_full

,2end,Upper,0,1,2,3,4,5,6,7,...,нг,йн,вь,дж,ба,зу,count_punctuations,Vow,noVow,has_digit
0,ен,True,0.043358,0.083087,0.545782,-0.652713,0.167219,1.041286,0.452239,0.319285,...,0,0,0,0,0,0,0,4,4,0
1,ар,True,0.723241,-0.220232,0.157270,-0.530469,-0.370502,1.080097,0.008935,-0.295212,...,0,0,0,0,0,0,0,2,1,0
2,он,True,0.563713,0.044549,0.729276,-0.006370,0.014865,0.863644,0.006354,-0.448426,...,0,0,0,0,0,0,0,3,2,0
3,он,False,1.243271,-0.661567,0.629706,0.034496,-0.061604,0.917440,0.180427,-0.409605,...,0,0,0,0,0,0,0,3,2,0
4,на,True,0.733357,-0.205593,0.257248,-0.535895,-0.136550,1.072279,-0.090591,-0.538158,...,0,0,0,0,0,0,0,4,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101403,ин,True,0.119514,0.046759,0.299056,-0.390972,0.132235,0.933751,0.067437,0.194212,...,0,0,0,0,0,0,0,1,4,0
101404,ой,False,-0.099496,-0.619223,0.420050,-0.495831,0.190697,0.712462,-0.632180,0.371763,...,0,0,0,0,0,0,0,1,4,0
101405,ок,False,1.323710,0.474011,1.024830,-0.018224,-0.554393,0.721366,0.557318,-0.045137,...,0,0,0,0,0,0,0,1,3,0
101406,ой,False,1.510883,0.508476,0.666330,-0.034387,-0.647684,0.841889,0.178684,-0.375572,...,0,0,0,0,0,0,0,2,5,0


In [35]:
cat_features = np.where(train_full.dtypes == 'object')[0].tolist()
train_full.columns[cat_features]

Index(['2end'], dtype='object')

In [36]:
cat_features

[0]

In [37]:
ctb = CatBoostClassifier(random_seed=17, iterations=800,
                         loss_function='CrossEntropy', eval_metric='AUC', thread_count=8)
ctb.fit(train_full, train['Label'], cat_features=cat_features)

Learning rate set to 0.075542
0:	total: 163ms	remaining: 2m 9s
1:	total: 238ms	remaining: 1m 34s
2:	total: 301ms	remaining: 1m 20s
3:	total: 367ms	remaining: 1m 12s
4:	total: 431ms	remaining: 1m 8s
5:	total: 497ms	remaining: 1m 5s
6:	total: 561ms	remaining: 1m 3s
7:	total: 627ms	remaining: 1m 2s
8:	total: 694ms	remaining: 1m
9:	total: 756ms	remaining: 59.7s
10:	total: 821ms	remaining: 58.9s
11:	total: 887ms	remaining: 58.2s
12:	total: 944ms	remaining: 57.1s
13:	total: 1.01s	remaining: 56.5s
14:	total: 1.06s	remaining: 55.5s
15:	total: 1.12s	remaining: 54.7s
16:	total: 1.16s	remaining: 53.5s
17:	total: 1.22s	remaining: 52.8s
18:	total: 1.27s	remaining: 52.1s
19:	total: 1.32s	remaining: 51.5s
20:	total: 1.36s	remaining: 50.6s
21:	total: 1.42s	remaining: 50s
22:	total: 1.47s	remaining: 49.8s
23:	total: 1.53s	remaining: 49.4s
24:	total: 1.59s	remaining: 49.2s
25:	total: 1.64s	remaining: 48.8s
26:	total: 1.7s	remaining: 48.7s
27:	total: 1.75s	remaining: 48.3s
28:	total: 1.8s	remaining: 48s


240:	total: 18.8s	remaining: 43.6s
241:	total: 18.9s	remaining: 43.6s
242:	total: 19s	remaining: 43.5s
243:	total: 19.1s	remaining: 43.5s
244:	total: 19.2s	remaining: 43.4s
245:	total: 19.2s	remaining: 43.3s
246:	total: 19.3s	remaining: 43.3s
247:	total: 19.4s	remaining: 43.2s
248:	total: 19.5s	remaining: 43.2s
249:	total: 19.6s	remaining: 43.1s
250:	total: 19.7s	remaining: 43s
251:	total: 19.8s	remaining: 43s
252:	total: 19.9s	remaining: 43s
253:	total: 20s	remaining: 42.9s
254:	total: 20.1s	remaining: 42.9s
255:	total: 20.1s	remaining: 42.8s
256:	total: 20.2s	remaining: 42.8s
257:	total: 20.3s	remaining: 42.7s
258:	total: 20.4s	remaining: 42.7s
259:	total: 20.5s	remaining: 42.6s
260:	total: 20.6s	remaining: 42.5s
261:	total: 20.7s	remaining: 42.5s
262:	total: 20.8s	remaining: 42.4s
263:	total: 20.9s	remaining: 42.4s
264:	total: 21s	remaining: 42.3s
265:	total: 21.1s	remaining: 42.3s
266:	total: 21.1s	remaining: 42.2s
267:	total: 21.2s	remaining: 42.2s
268:	total: 21.3s	remaining: 42.

477:	total: 41.2s	remaining: 27.8s
478:	total: 41.3s	remaining: 27.7s
479:	total: 41.4s	remaining: 27.6s
480:	total: 41.5s	remaining: 27.6s
481:	total: 41.6s	remaining: 27.5s
482:	total: 41.7s	remaining: 27.4s
483:	total: 41.8s	remaining: 27.3s
484:	total: 41.9s	remaining: 27.2s
485:	total: 42s	remaining: 27.1s
486:	total: 42.1s	remaining: 27.1s
487:	total: 42.2s	remaining: 27s
488:	total: 42.3s	remaining: 26.9s
489:	total: 42.4s	remaining: 26.8s
490:	total: 42.5s	remaining: 26.7s
491:	total: 42.6s	remaining: 26.7s
492:	total: 42.7s	remaining: 26.6s
493:	total: 42.8s	remaining: 26.5s
494:	total: 42.9s	remaining: 26.4s
495:	total: 43s	remaining: 26.4s
496:	total: 43.1s	remaining: 26.3s
497:	total: 43.2s	remaining: 26.2s
498:	total: 43.2s	remaining: 26.1s
499:	total: 43.3s	remaining: 26s
500:	total: 43.4s	remaining: 25.9s
501:	total: 43.5s	remaining: 25.8s
502:	total: 43.6s	remaining: 25.7s
503:	total: 43.7s	remaining: 25.6s
504:	total: 43.8s	remaining: 25.6s
505:	total: 43.8s	remaining:

716:	total: 1m 2s	remaining: 7.22s
717:	total: 1m 2s	remaining: 7.14s
718:	total: 1m 2s	remaining: 7.05s
719:	total: 1m 2s	remaining: 6.96s
720:	total: 1m 2s	remaining: 6.88s
721:	total: 1m 2s	remaining: 6.79s
722:	total: 1m 2s	remaining: 6.7s
723:	total: 1m 3s	remaining: 6.61s
724:	total: 1m 3s	remaining: 6.53s
725:	total: 1m 3s	remaining: 6.44s
726:	total: 1m 3s	remaining: 6.35s
727:	total: 1m 3s	remaining: 6.27s
728:	total: 1m 3s	remaining: 6.18s
729:	total: 1m 3s	remaining: 6.09s
730:	total: 1m 3s	remaining: 6s
731:	total: 1m 3s	remaining: 5.92s
732:	total: 1m 3s	remaining: 5.83s
733:	total: 1m 3s	remaining: 5.74s
734:	total: 1m 3s	remaining: 5.66s
735:	total: 1m 4s	remaining: 5.57s
736:	total: 1m 4s	remaining: 5.48s
737:	total: 1m 4s	remaining: 5.4s
738:	total: 1m 4s	remaining: 5.31s
739:	total: 1m 4s	remaining: 5.22s
740:	total: 1m 4s	remaining: 5.14s
741:	total: 1m 4s	remaining: 5.05s
742:	total: 1m 4s	remaining: 4.96s
743:	total: 1m 4s	remaining: 4.88s
744:	total: 1m 4s	remaini

In [38]:
# функция принимает на вход список фамилий и ответов и записывает файл


def write_ans(model_name, fam_list, ans):

    with open(model_name + '_answers.txt', 'w', encoding='utf8') as file_handler:
        file_handler.write('Id,Answer\n')
        for i in range(len(ans)):
            file_handler.write(str(i) + ',' + str(ans[i]) + '\n')
    file_handler.close()

In [39]:
write_ans('catboost', test_full, ctb.predict_proba(test_full)[:, 1])